In [1]:
import os
import pandas as pd
import nltk
nltk.download('punkt')
from datasets import load_dataset
from nltk import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /home/psachin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#first the one million jokes dataset to be marked as jokes
jokes = pd.read_csv('one-million-reddit-jokes.csv')
jokes.head()

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
0,post,ftbp1i,2qh72,jokes,False,1585785543,https://old.reddit.com/r/Jokes/comments/ftbp1i...,self.jokes,NaN,My corona is covered with foreskin so it is no...,I am soooo glad I'm not circumcised!,2
1,post,ftboup,2qh72,jokes,False,1585785522,https://old.reddit.com/r/Jokes/comments/ftboup...,self.jokes,NaN,It's called Google Sheets.,Did you know Google now has a platform for rec...,9
2,post,ftbopj,2qh72,jokes,False,1585785508,https://old.reddit.com/r/Jokes/comments/ftbopj...,self.jokes,NaN,The vacuum doesn't snore after sex.\n\n&amp;#x...,What is the difference between my wife and my ...,15
3,post,ftbnxh,2qh72,jokes,False,1585785428,https://old.reddit.com/r/Jokes/comments/ftbnxh...,self.jokes,NaN,[removed],My last joke for now.,9
4,post,ftbjpg,2qh72,jokes,False,1585785009,https://old.reddit.com/r/Jokes/comments/ftbjpg...,self.jokes,NaN,[removed],The Nintendo 64 turns 18 this week...,134


In [3]:
cols_to_drop = [column for column in jokes.columns if column not in ['selftext', 'title']]
jokes.drop(columns=cols_to_drop, axis = 1, inplace=True)
jokes.head()

,selftext,title
0,My corona is covered with foreskin so it is no...,I am soooo glad I'm not circumcised!
1,It's called Google Sheets.,Did you know Google now has a platform for rec...
2,The vacuum doesn't snore after sex.\n\n&amp;#x...,What is the difference between my wife and my ...
3,[removed],My last joke for now.
4,[removed],The Nintendo 64 turns 18 this week...


In [4]:
jokes = jokes[jokes['selftext'] != '[removed]']
jokes = jokes[jokes['title'] != '[removed]']
jokes.head()

,selftext,title
0,My corona is covered with foreskin so it is no...,I am soooo glad I'm not circumcised!
1,It's called Google Sheets.,Did you know Google now has a platform for rec...
2,The vacuum doesn't snore after sex.\n\n&amp;#x...,What is the difference between my wife and my ...
7,Oo..lala...,What did the French man say to the attractive ...
10,"Yo momma's so fat, that when she went to the z...",Yo Mama


In [5]:
jokes['whole_joke'] = jokes['title'] + ' ' +jokes['selftext']
jokes.head()

,selftext,title,whole_joke
0,My corona is covered with foreskin so it is no...,I am soooo glad I'm not circumcised!,I am soooo glad I'm not circumcised! My corona...
1,It's called Google Sheets.,Did you know Google now has a platform for rec...,Did you know Google now has a platform for rec...
2,The vacuum doesn't snore after sex.\n\n&amp;#x...,What is the difference between my wife and my ...,What is the difference between my wife and my ...
7,Oo..lala...,What did the French man say to the attractive ...,What did the French man say to the attractive ...
10,"Yo momma's so fat, that when she went to the z...",Yo Mama,"Yo Mama Yo momma's so fat, that when she went ..."


In [6]:
#cnn daily mail datasets, gotten from huggingface datasets
splits = ['train', 'test', 'validation']
highlights = []
for split in splits:
    split_bit = load_dataset('cnn_dailymail', '3.0.0', split=split)
    for row in split_bit:
        if 'highlights' in row:
            highlights.append(row['highlights'])

Reusing dataset cnn_dailymail (/home/psachin/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)
Reusing dataset cnn_dailymail (/home/psachin/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)
Reusing dataset cnn_dailymail (/home/psachin/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


In [7]:
import string
table = str.maketrans('', '', string.punctuation)
stop_words = set(stopwords.words('english'))
def clean_data(doc):
    tokens = word_tokenize(doc)
    tokens = [w.lower() for w in tokens]
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

In [9]:
final = []
for doc in jokes['whole_joke'].astype('str'):
    #print('Trying for: ' + doc)
    cleaned = clean_data(doc)
    final.append([doc, cleaned, 1])

for doc in highlights:
    cleaned = clean_data(doc)
    final.append([doc, cleaned, -1])

In [10]:
final = pd.DataFrame(final, columns=['original text', 'cleaned text', 'joke'])
final.head()

,original text,cleaned text,joke
0,I am soooo glad I'm not circumcised! My corona...,soooo glad circumcised corona covered foreskin...,1
1,Did you know Google now has a platform for rec...,know google platform recording bowel movements...,1
2,What is the difference between my wife and my ...,difference wife vacuum vacuum nt snore sex amp...,1
3,What did the French man say to the attractive ...,french man say attractive teletubby oo lala,1
4,"Yo Mama Yo momma's so fat, that when she went ...",yo mama yo momma fat went zoo hippos got jealous,1


In [11]:
final.to_csv('cleaned_data.csv')

In [18]:
final.iloc[0, 0]

"I am soooo glad I'm not circumcised! My corona is covered with foreskin so it is not exposed to viruses."